# Basic Data Extraction from Vissim 7 Simulation
* 본 코드의 기능
    * 특정 네트워크를 바탕으로 VISSIM 7에서 시뮬레이션 실행
    * 네트워크로부터 각종 필수 데이터를 추출하여 .csv 파일로 저장

# Imports

In [1]:
import win32com.client as com

from tqdm import tqdm
import os

import numpy as np
import pandas as pd

# Load VISSIM file

## Open a new Vissim Window
* COM Server 연결 => 새로운 VISSIM 창 열기(VISSIM이 실행됨)

In [2]:
Vissim = com.Dispatch('Vissim.Vissim-64.700') # Vissim = com.Dispatch() # Vissim 7 - 64 bit - VISSIM을 연다

## Load a Vissim Network & Layout

In [3]:
network_folder_path = 'D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Personal/03_VISSIM_win32com/network_example'

In [4]:
read_inpx_name = 'COM_example.inpx'
read_inpx_path = os.path.abspath(os.path.join(network_folder_path, read_inpx_name)) # 네트워크 파일인 .inpx 로드

print(read_inpx_path)

flag_read_additionally = False # you can read network(elements) additionally, in this case set "flag_read_additionally" to true
Vissim.LoadNet(read_inpx_path, flag_read_additionally) # 해당 파일을 오픈

D:\OneDrive - 연세대학교 (Yonsei University)\Projects\Personal\03_VISSIM_win32com\network_example\COM_example.inpx


In [5]:
read_layx_name = 'COM_example.layx'
read_layx_path = os.path.join(network_folder_path, read_layx_name) # 레이아웃 .layx 로드

print(read_layx_path)

Vissim.LoadLayout(read_layx_path)

D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Personal/03_VISSIM_win32com/network_example\COM_example.layx


# Extract Data
* Note: All of the following commands can also be executed during a simulation.

In [6]:
data_folder_path = 'D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Personal/03_VISSIM_win32com/network_example/data'

os.makedirs(data_folder_path, exist_ok = True)

## Links list

In [82]:
link_att_list = ['No', 'Name', 'LinkBehavType', 'Level', 'NumLanes', 'Length2D']

for i in range(len(link_att_list)):
    globals()[f'link_tup_{i}'] = Vissim.Net.Links.GetMultiAttValues(link_att_list[i])
    
    link_local_df = pd.DataFrame.from_records(globals()[f'link_tup_{i}'], columns = ['Count', link_att_list[i]])
    
    if i == 0:
        link_list_df = link_local_df
    
    else :
        link_list_df = pd.merge(link_list_df, link_local_df, how = 'left', on = 'Count')

save_link_name = 'links_list.csv'
save_link_path = os.path.join(data_folder_path, save_link_name)

link_list_df.to_csv(save_link_path, encoding = 'cp949')

## Veicle Inputs

In [83]:
VI_att_list = ['No', 'Name', 'Link', 'Volume(1)', 'Volume(2)', 'VehComp(1)', 'VehComp(2)']

for i in range(len(VI_att_list)):
    globals()[f'VI_tup_{i}'] = Vissim.Net.VehicleInputs.GetMultiAttValues(VI_att_list[i])
    
    VI_local_df = pd.DataFrame.from_records(globals()[f'VI_tup_{i}'], columns = ['Count', VI_att_list[i]])
    
    if i == 0:
        VI_list_df = VI_local_df
    
    else :
        VI_list_df = pd.merge(VI_list_df, VI_local_df, how = 'left', on = 'Count')

save_VI_name = 'VI_list.csv'
save_VI_path = os.path.join(data_folder_path, save_VI_name)

VI_list_df.to_csv(save_VI_path, encoding = 'cp949')

## Vehicle Routes(Static)
* SVRD : 각 Vehicle Route 시작위치 정보
* SVR : 각 Vehicle Route에서 방향별 종료위치, Relative Flow 정보 기록

In [11]:
SVRD_att_list = ['No', 'Name', 'Link', 'Pos']

# SVRD 리스트 뽑기
for i in range(len(SVRD_att_list)):
    globals()[f'SVRD_tup_{i}'] = Vissim.Net.VehicleRoutingDecisionsStatic.GetMultiAttValues(SVRD_att_list[i])
    
    SVRD_local_df = pd.DataFrame.from_records(globals()[f'SVRD_tup_{i}'], columns = ['Count', SVRD_att_list[i]])
    
    if i == 0:
        SVRD_list_df = SVRD_local_df
    
    else :
        SVRD_list_df = pd.merge(SVRD_list_df, SVRD_local_df, how = 'left', on = 'Count')

save_SVRD_name = 'SVRD_list.csv'
save_SVRD_path = os.path.join(data_folder_path, save_SVRD_name)
SVRD_list_df.to_csv(save_SVRD_path, encoding = 'cp949')

In [83]:
SVR_att_list = ['No', 'Name', 'DestLink', 'DestPos', 'RelFlow(1)']
SVRD_num_list = SVRD_list_df['No']

# 각 SVRD 별 SVR 리스트 뽑기
SVR_local_df = []
SVR_num_df = []
SVR_total_df = pd.DataFrame()

for num in SVRD_num_list:
    
    for i in range(len(SVR_att_list)):
        globals()[f'SVR_{num}_{i}'] = Vissim.Net.VehicleRoutingDecisionsStatic.ItemByKey(num).VehRoutSta.GetMultiAttValues(SVR_att_list[i])
    
        SVR_local_df = pd.DataFrame.from_records(globals()[f'SVR_{num}_{i}'], columns = ['Count', SVR_att_list[i]])
        SVR_local_df.insert(1, 'SVDR', num, allow_duplicates = False)
    
        if i == 0:
            SVR_num_df = SVR_local_df
    
        else :
            SVR_num_df = pd.merge(SVR_num_df, SVR_local_df, how = 'left', on = ['Count', 'SVDR'])
            
    SVR_total_df = pd.concat([SVR_total_df, SVR_num_df], axis = 0)
    
save_SVR_name = 'SVR_list.csv'
save_SVR_path = os.path.join(data_folder_path, save_SVR_name)
SVR_total_df.to_csv(save_SVR_path, encoding = 'cp949')

In [84]:
SVR_total_df

,Count,SVDR,No,Name,DestLink,DestPos,RelFlow(1)
0,1,1,1,,5,128.445706,0.6
1,2,1,2,,11,264.901440,0.5
0,1,2,1,,11,263.027664,0.5
1,2,2,2,,14,212.136061,0.5
0,1,3,1,,14,212.984181,0.5
1,2,3,2,,5,125.930510,0.5


## Signal Heads

## Vehicle Travel Times

## Vehicle Composition


# Accessing Multiple Attributes
* 모든 링크 번호, ... 등 출력하기

In [103]:
# SetAllAttValues           Set all attributes of one object to one value:
Attribute = "Name"
Link_Name = "All Links have the same Name"
Vissim.Net.Links.SetAllAttValues(Attribute, Link_Name)
Attribute = "CostPerKm"
Cost = 5.5
Vissim.Net.Links.SetAllAttValues(Attribute, Cost)
# Note the method SetAllAttValues has a 3rd optional input: Optional ByVal add As Boolean = False; Use only for numbers!
Vissim.Net.Links.SetAllAttValues(Attribute, Cost, True) # setting the 3rd input to true, will add 5.5 to all previous costs!